Import library

In [2]:
import pandas as pd
import numpy as np
import re
from sklearn.utils import shuffle

Load data

In [13]:
real=pd.read_csv('./True.csv')
fake=pd.read_csv('./Fake.csv')


In [14]:
real.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [15]:
fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [22]:
real['value']=1
fake['value']=0

data=pd.concat([true,fake],ignore_index=True)
data=shuffle(data)

In [23]:
data.sample(10)

,title,text,subject,date,value
17784,New Afghan peace talks expected in Oman but Ta...,"KABUL/PESHAWAR, Pakistan (Reuters) - Represent...",worldnews,"October 11, 2017",1
10931,"Obama, Italy's Mattarella discuss fight agains...",WASHINGTON (Reuters) - U.S. President Barack O...,politicsNews,"February 8, 2016",1
10738,Republican Marco Rubio winning Wall Street fun...,WASHINGTON/NEW YORK (Reuters) - Republican Mar...,politicsNews,"February 22, 2016",1
26272,Maine GOP State Senator Sick Of His Bullsh*t ...,Republicans in Maine tolerated Governor Paul L...,News,"August 29, 2016",0
38883,EMBARRASSING: Anti-Trump “THE HILL” Gets SLAMM...,"#JokeNewsAfter record, cold temperatures were ...",left-news,"Dec 30, 2017",0
44106,#Hashtag Hell & The Fake Left,By Dady Chery and Gilbert MercierAll writers ...,US_News,"January 19, 2016",0
5987,Trump administration debates designating Musli...,WASHINGTON (Reuters) - A debate is under way i...,politicsNews,"January 29, 2017",1
23238,Anne Frank Center SAVAGES Spicer After He Sai...,It s no secret that a large percentage of the ...,News,"April 11, 2017",0
44019,Utah Ranchers Vow to Stand Up to Government Ab...,"Sam Levin The GuardianOn 23 January, a group o...",US_News,"April 5, 2016",0
5021,Trump invites Palestinian leader Abbas to Whit...,JERUSALEM (Reuters) - U.S. President Donald Tr...,politicsNews,"March 10, 2017",1


In [24]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44898 entries, 12602 to 39133
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    44898 non-null  object
 1   text     44898 non-null  object
 2   subject  44898 non-null  object
 3   date     44898 non-null  object
 4   value    44898 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 2.1+ MB


In [25]:
data.drop(columns=['subject','date'],axis=1,inplace=True)

In [26]:
data.head()

,title,text,value
12602,"Britain committed to no hard Irish border, wha...",LONDON (Reuters) - Britain intends to prevent ...,1
31097,CLINTON FAMILY REFUSES TO RETURN ‘Dirty’ Harve...,After Bill and Chelsea Clinton both dodged re...,0
28541,Megyn Kelly Laughably Claims Fox News Does No...,You ll have a hard time not laughing out loud ...,0
37444,JPMORGAN CEO Blows Up at the DC Dysfunction: T...,JPMorgan Chase CEO Jamie Dimon went on a rant ...,0
4850,White House tax reform may begin in late sprin...,DUBLIN (Reuters) - President Donald Trump may ...,1


In [31]:
data['text'][100]

'WASHINGTON (Reuters) - Democratic Senator Elizabeth Warren is taking aim at budget chief Mick Mulvaney’s plan to fill the ranks of the U.S. consumer financial watchdog with political allies, according to letters seen by Reuters, the latest salvo in a broader battle over who should run the bureau. President Donald Trump last month appointed Mulvaney as acting director of the Consumer Financial Protection Bureau (CFPB), though the decision is being legally challenged by the agency’s deputy director, Leandra English, who says she is the rightful interim head. Mulvaney told reporters earlier this month he planned to bring in several political appointees to help overhaul the agency, but Warren warned in a pair of letters sent Monday to Mulvaney and the Office of Personnel Management (OPM), which oversees federal hiring, that doing so was inappropriate and potentially illegal. The CFPB is meant to be an independent agency staffed primarily by non-political employees. Hiring political appoin

**Remove RegEx**

In [70]:
data['processed']=data['text'].map(lambda x: re.sub('(Reuters)','',x))
data['processed']=data['processed'].map(lambda x: re.sub('[^A-Za-z0-9]+',' ',x))
data['processed']=data['processed'].map(lambda x: x.lower())

In [71]:
data['processed'][20]

'washington a federal judge in seattle partially blocked u s president donald trump s newest restrictions on refugee admissions on saturday the latest legal defeat for his efforts to curtail immigration and travel to the united states the decision by u s district judge james robart is the first judicial curb on rules the trump administration put into place in late october that have contributed significantly to a precipitous drop in the number of refugees being admitted into the country refugees and groups that assist them argued in court that the administration s policies violated the constitution and federal rulemaking procedures among other claims department of justice attorneys argued in part that u s law grants the executive branch the authority to limit refugee admissions in the way that it had done so on oct 24 the trump administration effectively paused refugee admissions from 11 countries mostly in the middle east and africa pending a 90 day security review which was set to exp

In [72]:
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [73]:
def lemmatize(text):
  lm=WordNetLemmatizer()
  tokens=[lm.lemmatize(word) for word in text.split()]
  return ' '.join(tokens)

In [74]:
lemmatize(data['processed'][20])

'washington a federal judge in seattle partially blocked u s president donald trump s newest restriction on refugee admission on saturday the latest legal defeat for his effort to curtail immigration and travel to the united state the decision by u s district judge james robart is the first judicial curb on rule the trump administration put into place in late october that have contributed significantly to a precipitous drop in the number of refugee being admitted into the country refugee and group that assist them argued in court that the administration s policy violated the constitution and federal rulemaking procedure among other claim department of justice attorney argued in part that u s law grant the executive branch the authority to limit refugee admission in the way that it had done so on oct 24 the trump administration effectively paused refugee admission from 11 country mostly in the middle east and africa pending a 90 day security review which wa set to expire in late january

In [75]:
data['processed']=data['processed'].apply(lemmatize)

**Frequency Distribution**

In [126]:
from nltk import FreqDist

In [131]:
def frequency(text):
  lm=WordNetLemmatizer()
  token=[lm.lemmatize(word) for word in text]
  frequency_distribution = FreqDist(token)

In [132]:
data['processed']=data['processed'].apply(frequency)

In [ ]:
frequency_distribution.most_common(20)

**Make TfidfVectorizer**

In [76]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [108]:
Y=data['value']
X=data['processed']
x_train,x_test,y_train,y_test=train_test_split(X,Y,random_state=100,stratify=Y, test_size=0.2)

In [111]:
tfidf=TfidfVectorizer(stop_words='english',ngram_range=(1,3),lowercase=True,max_features=5000)
x_train_processed=tfidf.fit_transform(x_train)
x_test_processed=tfidf.fit_transform(x_test)

In [112]:
x_train.shape

(35918,)

**PassiveAggressiveClassifier**

In [136]:
from sklearn.linear_model import PassiveAggressiveClassifier

In [138]:
psc=PassiveAggressiveClassifier(max_iter=100)
psc.fit(x_train_processed, y_train)

PassiveAggressiveClassifier(C=1.0, average=False, class_weight=None,
                            early_stopping=False, fit_intercept=True,
                            loss='hinge', max_iter=100, n_iter_no_change=5,
                            n_jobs=None, random_state=None, shuffle=True,
                            tol=0.001, validation_fraction=0.1, verbose=0,
                            warm_start=False)

In [140]:
pred=psc.predict(x_test_processed)
accuracy_score(y_test,pred)

0.6337416481069043


Logistic Regression

In [113]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,confusion_matrix

In [114]:
lr = LogisticRegression()
lr.fit(x_train_processed, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [115]:
pred=lr.predict(x_test_processed)

In [116]:
accuracy_score(y_test,pred)

0.6005567928730512

In [117]:
confusion_matrix(y_test,pred)

array([[4514,  182],
       [3405,  879]])

**Decision Tree Classifier**

In [118]:

from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()
dt.fit(x_train_processed, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [119]:
pred=dt.predict(x_test_processed)

In [120]:
accuracy_score(y_test,pred)

0.5689309576837417

In [121]:
confusion_matrix(y_test,pred)

array([[2580, 2116],
       [1755, 2529]])

**Random Forest Classifier**

In [122]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
rfc.fit(x_train_processed, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [123]:
pred=rfc.predict(x_test_processed)

In [124]:
accuracy_score(y_test,pred)

0.6800668151447662

In [125]:
confusion_matrix(y_test,pred)

array([[4009,  687],
       [2186, 2098]])

Save Model

In [141]:
import pickle
pickle.dump(rfc,open('rfc_mode.pkl','wb'))